In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
import cv2
from keras_efficientnets import EfficientNetB5

print(tf.__version__)
print(tf.test.is_gpu_available())

# gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
# tf.config.experimental.set_virtual_device_configuration( gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15500)])
#Tesla P100-PCIE-16GB

def load_img(img,lab):
    img=tf.io.read_file(img)
    img=tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img,tf.float32) ##???
    lab = tf.cast(lab,tf.float32)/2 #####???    turn 0,2,4 to 0,1,2 otherwise loss=NaN
    return img,lab

test_data=pd.read_csv('test.txt',header=None,sep=' ',names=['picture','label'])
train_data=pd.read_csv('train.txt',header=None,sep=' ',names=['picture','label'])
valid_data=pd.read_csv('valid.txt',header=None,sep=' ',names=['picture','label'])

test_data['dir']=['preprocessed/'+'test/'+pic for pic in test_data['picture']]
train_data['dir']=['preprocessed/'+'train/'+pic for pic in train_data['picture']]
valid_data['dir']=['preprocessed/'+'valid/'+pic for pic in valid_data['picture']]

total_set = pd.concat([test_data,train_data,valid_data],ignore_index=True)
iid_total_set = pd.concat([total_set[total_set['label']==i*2][:1000] for i in range(3)])

random_set = iid_total_set.sample(frac=1).reset_index(drop=True)

## settings
fraction_val = 0.2 ############################################################################

valid_LEN=int(len(random_set)*fraction_val)
train_LEN=int(len(random_set)-valid_LEN)

batch_SIZE=5 ####################################################################################
train_EPO=int(train_LEN // batch_SIZE)
valid_EPO=int(valid_LEN // batch_SIZE)

iid_valid = random_set[:valid_LEN]
iid_train = random_set[valid_LEN:]

In [ ]:
train_SET = tf.data.Dataset.from_tensor_slices((iid_train['dir'],iid_train['label'])).\
                            shuffle(train_LEN).\
                            map(load_img).\
                            batch(batch_SIZE).\
                            repeat().\
                            prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_SET = tf.data.Dataset.from_tensor_slices((iid_valid['dir'],iid_valid['label'])).\
                            shuffle(valid_LEN).\
                            map(load_img).\
                            batch(batch_SIZE).\
                            repeat().\
                            prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
effnet = EfficientNetB5(weights='imagenet',
                        include_top=False,
                        input_shape=(456, 456, 3))
effnet.trainable = False

In [ ]:
import keras
def build_model(effnet):
    model = keras.models.Sequential()
    model.add(effnet)
    model.add(keras.layers.GlobalAveragePooling2D())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(5, activation='relu'))
    model.add(keras.layers.Dense(3))
    return model

# Initialize model
model = build_model(effnet)

In [ ]:
model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])
# model.summary()

In [ ]:
# model._get_trainable_state()

In [ ]:
epo=5
history= model.fit(np.array(train_SET),
                  epochs=epo,
                  steps_per_epoch=train_EPO,
                  validation_data=valid_SET,
                  validation_steps=valid_EPO
                  )

In [ ]:
model.trainable = True

In [ ]:
# model._get_trainable_state()

In [ ]:
epo=10
history= model.fit(train_SET,
                  epochs=epo,
                  steps_per_epoch=train_EPO,
                  validation_data=valid_SET,
                  validation_steps=valid_EPO
                  )

In [ ]:
plt.plot(history.epoch, history.history.get('acc'), label='acc')
plt.plot(history.epoch, history.history.get('val_acc'), label='val_acc')
plt.legend()

In [ ]:
plt.plot(history.epoch, history.history.get('loss'), label='loss')
plt.plot(history.epoch, history.history.get('val_loss'), label='val_loss')
plt.legend()

In [ ]:
# model.save_weights('myXception.h5')

# acc = history.history.get('acc')
# val_acc = history.history.get('val_acc')
# loss = history.history.get('loss')
# val_loss = history.history.get('val_loss')


# np.save("acc.npy", acc)
# np.save("val_acc.npy", val_acc)
# np.save("loss.npy", loss)
# np.save("val_loss.npy", val_loss)